In [1]:
import pandas as pd
import numpy as np
from pysummarization.nlpbase.auto_abstractor import AutoAbstractor
from pysummarization.tokenizabledoc.simple_tokenizer import SimpleTokenizer
from pysummarization.abstractabledoc.top_n_rank_abstractor import TopNRankAbstractor
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer

In [2]:
def rank(subtitle_file,excel_name,sheet_name):
    
    #read input file
    file = open(subtitle_file)
    SENTENCES_COUNT =0         
    data = file.read()

    CoList = data.split('\n')

    for i in CoList:
        if i:        
            SENTENCES_COUNT += 1

    data = data.replace('\n', ' . ')
    #close the input file
    file.close()

    ############
    #open the input file in write mode
    fin = open(subtitle_file, "wt")
    #overrite the input file with the resulting data
    fin.write(data)
    #close the file
    fin.close()

    ###### read again####
    document = open(subtitle_file).read()

    ####PANDAS####
    df = pd.read_excel(excel_name,header=0,sheet_name=sheet_name,index_col=0)
    df.index.name = None
    df =df.fillna(0)
    df= df.astype(int)
    

    #####PYSUMMM#####
    
    auto_abstractor = AutoAbstractor() # Object of automatic summarization.
    auto_abstractor.tokenizable_doc = SimpleTokenizer() # Set tokenizer.
    auto_abstractor.delimiter_list = ["."] # Set delimiter for making a list of sentence.
    abstractable_doc = TopNRankAbstractor()# Object of abstracting and filtering document.
    
    for n in range(1,SENTENCES_COUNT+1):
        abstractable_doc.set_top_n(n) # set n = 120 value
        result_dict = auto_abstractor.summarize(document, abstractable_doc) # Summarize document.
        #print(n )

        for i in range(n):
            a=result_dict["scoring_data"][i][0]+1
            #print('a ='+str(a))
            if df['pysum'][a]==0:
                df['pysum'][a] = n

    #####SUMY#########
    LANGUAGE = "english"
    parser = PlaintextParser.from_file(subtitle_file, Tokenizer(LANGUAGE))
    summarizer_lex = LexRankSummarizer()

    summarizer_luhn = LuhnSummarizer()

    summarizer_lsa = LsaSummarizer()

    summarizer_textrank = TextRankSummarizer()

    for n in range(1,SENTENCES_COUNT):
        summary_lex = summarizer_lex(parser.document,n)
        summary_luhn = summarizer_luhn(parser.document,n)
        summary_lsa = summarizer_lsa(parser.document,n)
        summary_textrank = summarizer_textrank(parser.document,n)
        for sentence in summary_lex:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['lexrank'][a]==0:
                df['lexrank'][a] = n
        for sentence in summary_luhn:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['luhn'][a]==0:
                df['luhn'][a] = n
        for sentence in summary_lsa:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['lsa'][a]==0:
                df['lsa'][a] = n
        for sentence in summary_textrank:        
            a=parser.document.sentences.index(sentence)+1
            #print('a ='+str(a))
            if df['textrank'][a]==0:
                df['textrank'][a] = n

    return df

In [3]:
rank(subtitle_file='Sub_1.txt',excel_name='data.xlsx',sheet_name='Sheet1')

,pysum,lexrank,luhn,lsa,textrank,x1.,x1.3,x1.6,x1.9,x2.2,x2.5,nadir_kelime,konu_ile_alakalı
1,8,78,63,109,88,1,1,1,1,1,1,0,0
2,11,64,102,24,83,1,1,1,1,1,1,0,0
3,10,71,83,27,106,1,1,1,1,1,0,0,0
4,7,110,7,18,12,1,1,1,1,1,1,0,0
5,6,2,3,17,4,1,1,1,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,99,84,97,115,0,1,1,1,1,1,1,0,0
117,49,25,21,20,43,1,1,1,1,1,1,0,0
118,39,31,32,16,16,1,1,1,1,1,0,0,0
119,24,38,33,1,1,1,1,1,1,1,0,0,0


In [15]:
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

In [18]:
documentA = open('Sub_1.txt').read()

In [20]:
bagOfWordsA = documentA.split(' ')
bagOfWordsA

['i',
 'was',
 'exactly',
 'where',
 'i',
 'wanted',
 'to',
 'be',
 'close',
 '.',
 'enough',
 'to',
 'kill',
 'assassin',
 'chou',
 'john',
 'was',
 '.',
 'trained',
 'by',
 'none',
 'other',
 'than',
 'ezio',
 'auditore',
 '.',
 'and',
 'she',
 'really',
 'wants',
 'a',
 'box',
 'the',
 'artifact',
 '.',
 'in',
 'the',
 'box',
 'is',
 'irrelevant',
 "it's",
 'just',
 'a',
 '.',
 'macguffin',
 'that',
 'sends',
 'her',
 'off',
 'on',
 'a',
 '2-d',
 '.',
 'stealth',
 'adventure',
 'called',
 "assassin's",
 '.',
 'creed',
 'chronicles',
 'china',
 'that',
 'looks',
 '.',
 'absolutely',
 'stunning',
 'but',
 'ultimately',
 'comes',
 '.',
 'off',
 'as',
 'a',
 'mildly',
 'enjoyable',
 'mark',
 'of',
 'the',
 '.',
 'ninja',
 'wannabe',
 'if',
 'a',
 'game',
 'must',
 'rest',
 'on',
 '.',
 'being',
 'pretty',
 'then',
 'at',
 'least',
 'chronicles',
 '.',
 'makes',
 'the',
 'most',
 'of',
 'it',
 'gorgeous',
 'moon',
 'rises',
 '.',
 'and',
 'lush',
 'cherry',
 'blossoms',
 'have',
 'the',


In [21]:
uniqueWords = set(bagOfWordsA)

In [22]:
numOfWordsA = dict.fromkeys(uniqueWords, 0)
for word in bagOfWordsA:
    numOfWordsA[word] += 1
    


In [23]:
stopwords. words('english')

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [24]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [25]:
tfA = computeTF(numOfWordsA, bagOfWordsA)

In [26]:
def computeIDF(documents):
    import math
    N = len(documents)
    
    idfDict = dict.fromkeys(documents[0].keys(), 0)
    for document in documents:
        for word, val in document.items():
            if val > 0:
                idfDict[word] += 1
    
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [27]:
idfs = computeIDF([numOfWordsA, numOfWordsA])

In [28]:
def computeTFIDF(tfBagOfWords, idfs):
    tfidf = {}
    for word, val in tfBagOfWords.items():
        tfidf[word] = val * idfs[word]
    return tfidf

In [29]:
tfidfA = computeTFIDF(tfA, idfs)

In [30]:
df = pd.DataFrame([tfidfA])

In [32]:
df.head(5)

,swing,toss,world,engage,ledge,this,like,dart,flaws,finest,...,fending,trained,hour,any,of,creed,me,eerie,auditore,blade
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([documentA])
feature_names = vectorizer.get_feature_names()
dense = vectors.todense()
denselist = dense.tolist()
df = pd.DataFrame(denselist, columns=feature_names)

In [35]:
df

,20,2d,above,absolutely,accustomed,achieve,across,add,adventure,affair,...,while,will,with,work,world,would,xiao,yet,you,your
0,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,...,0.014006,0.056022,0.042017,0.014006,0.014006,0.014006,0.028011,0.028011,0.420166,0.098039


In [36]:
df['Minimum'] = df.loc[:, :].min(axis=1)

In [37]:
df

,20,2d,above,absolutely,accustomed,achieve,across,add,adventure,affair,...,will,with,work,world,would,xiao,yet,you,your,Minimum
0,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,0.014006,...,0.056022,0.042017,0.014006,0.014006,0.014006,0.028011,0.028011,0.420166,0.098039,0.014006
